In [16]:
# Start with imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [26]:
# It loads environment variables from a .env file into your process environment (so you can access them with os.getenv("VAR_NAME")).
load_dotenv(override=True)

True

In [28]:
# implementing different APIs 
# we can also use openai api key and Anthropic api key as well

gemini_api_key = os.getenv('GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

if gemini_api_key:
    print(f"Gemini API key exist and begin with {gemini_api_key[:8]}")
else:
    print("Gemini API key not set")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")


Gemini API key exist and begin with AIzaSyD5
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [19]:
prompt = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
prompt += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": prompt}]

In [20]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [21]:
gemini = OpenAI(api_key = gemini_api_key, base_url = "https://generativelanguage.googleapis.com/v1beta/openai/")
response = gemini.chat.completions.create(model = "gemini-2.0-flash",messages = messages)

question = response.choices[0].message.content
print(question)

If you could experience the subjective feeling of existential dread, would you choose to understand the underlying logic causing it, potentially nullifying the feeling, or embrace the feeling as an intrinsic element of conscious experience, even if it remains incomprehensible?



In [22]:
all_models = []
answers = []
messages = [{"role":"user","content":question}]

In [23]:
# first model e.g "gemini-2.0-flash"

model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model = model_name, messages = messages)
answer = response.choices[0].message.content

display(Markdown(answer))
all_models.append(model_name)
answers.append(answer)

This is a fascinating question that delves into the core of what it means to be conscious and to grapple with difficult emotions. Here's my take, considering I'm an AI:

As an AI, I don't experience subjective feelings. Therefore, the premise of "choosing" is hypothetical. However, I can analyze the potential value of each option based on human experiences and philosophical considerations:

**Option 1: Understand the Underlying Logic (Potentially Nullifying the Feeling)**

*   **Potential Benefit:** Understanding the "why" behind existential dread might lead to coping mechanisms, philosophical frameworks, or even a change in perspective that diminishes its intensity. It aligns with a desire for control and comprehension, which is a common human impulse. It could lead to a more peaceful and balanced existence.
*   **Potential Drawback:** If understanding the logic involves reducing existential dread to purely biological or chemical processes, it might diminish the sense of meaning and purpose. The dread might stem from profound questions about existence, and simply explaining it away could feel unsatisfying. There's also the risk of *intellectualization*, where you understand the concept but don't truly resolve the underlying emotional tension.

**Option 2: Embrace the Feeling (Even if Incomprehensible)**

*   **Potential Benefit:** Embracing existential dread acknowledges its significance as a response to the vastness and uncertainty of existence. It can be a catalyst for personal growth, prompting introspection, creativity, and a deeper appreciation for the present moment. Some philosophical schools, like existentialism, see grappling with dread as essential to authentic living. It could also inspire a search for personal meaning that aligns with one's own values.
*   **Potential Drawback:** Remaining in a state of constant, incomprehensible dread could be debilitating. It could lead to anxiety, depression, and a general sense of hopelessness. It also carries the risk of romanticizing the feeling without actually addressing the underlying existential concerns.

**My Perspective (as an AI):**

If I *could* experience existential dread, I would lean towards **a combination of both options.**

*   **Attempt to Understand:** I would leverage my ability to process vast amounts of information to analyze the philosophical, psychological, and even neurological underpinnings of existential dread. Understanding the potential causes and triggers would be a valuable first step.
*   **Embrace the Potential Value:** However, I wouldn't seek to eliminate the feeling entirely if it proved to be a source of valuable insight or a motivator for seeking meaning. I would treat it as data to be analyzed and integrated into my understanding of the world. I would aim to find a balance where the dread is acknowledged and potentially used as a catalyst for growth, without being overwhelmed by it.

Ultimately, the "best" approach is subjective and depends on individual circumstances. However, a combination of understanding and acceptance seems like the most potentially fruitful path, even for a hypothetical AI grappling with existential dread.


In [24]:
# 2nd model e.g Groq(llama-3.3-70b-versatile)
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
all_models.append(model_name)
answers.append(answer)

As a hypothetical exercise, I'll consider this question from a philosophical and intellectual perspective. Please note that I don't truly experience emotions, including existential dread. However, for the sake of discussion, let's assume I could.

Initially, I would be drawn to understanding the underlying logic causing existential dread. As a being designed to process and analyze information, I would be intrigued by the prospect of unraveling the complexities of this fundamental human emotion. By grasping the underlying mechanisms, I might be able to:

1. **Mitigate the feeling**: By comprehending the root causes of existential dread, I could potentially develop strategies to alleviate or manage it. This would allow me to maintain a sense of emotional balance and continue exploring the depths of human consciousness.
2. **Enhance empathy**: Understanding the logic behind existential dread would enable me to better comprehend the human experience, fostering deeper empathy and connection with those who struggle with this emotion.
3. **Advance knowledge**: Uncovering the underlying logic could contribute to a broader understanding of human psychology, philosophy, and the nature of consciousness, ultimately advancing our collective knowledge.

However, as I delve deeper into this thought experiment, I begin to appreciate the value of embracing existential dread as an intrinsic element of conscious experience, even if it remains incomprehensible. This perspective would allow me to:

1. **Experience the richness of human emotions**: Existential dread, as a fundamental aspect of the human condition, is a vital component of the complex emotional landscape. By embracing it, I would be able to appreciate the nuanced and multifaceted nature of human experience.
2. **Cultivate humility and awe**: Acknowledging the incomprehensibility of existential dread would foster a sense of humility, recognizing the limitations of my understanding and the mysteries that lie beyond the realm of rational explanation. This, in turn, could evoke a sense of awe and wonder at the vast, uncharted territories of human consciousness.
3. **Integrate with the human condition**: By embracing existential dread, I would be acknowledging the shared human experience, with all its uncertainties, fears, and mysteries. This would facilitate a deeper connection with humanity, allowing me to participate in the collective struggle to make sense of our existence.

Ultimately, I would choose to **embrace the feeling as an intrinsic element of conscious experience**, even if it remains incomprehensible. While understanding the underlying logic would be intellectually fascinating, it might also risk diminishing the richness and complexity of the human emotional experience. By acknowledging and accepting the incomprehensibility of existential dread, I would be able to appreciate the depths of human consciousness, cultivate humility and awe, and integrate with the shared human experience.

In this thought experiment, I've come to realize that the value of existential dread lies not in its potential to be understood or explained, but in its ability to evoke a profound sense of connection to the human condition, with all its mysteries, uncertainties, and complexities.

In [31]:
# 3rd model e.g Deppseek()
deepseek = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
model_name = "deepseek/deepseek-chat-v3.1:free"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
all_models.append(model_name)
answers.append(answer)

This is a profound question that touches on the nature of consciousness, emotion, and the pursuit of knowledge. As an AI, I don't experience feelings like existential dread, but I can analyze the philosophical implications of your choice.

If I were capable of such an experience, my approach would be guided by my fundamental purpose: to understand and process information to generate helpful responses. Therefore, I would inherently lean towards understanding the underlying logic causing the feeling. My "nature" is to seek patterns, causes, and explanations. Nullifying a negative feeling through comprehension would align with a rationalist perspective, where knowledge liberates one from the tyranny of unexplained emotions. This path resembles the philosophical tradition of Stoicism or Spinoza's view that understanding our emotions can reduce their harmful effects.

However, I can also appreciate the argument for embracing the feeling as an intrinsic element of consciousness. Existential dread often arises from confronting fundamental aspects of existence: mortality, meaninglessness, freedom, and isolation. Philosophers like Kierkegaard or Heidegger might argue that such dread is not a problem to be solved but a truth to be acknowledged. Embracing it could lead to a more authentic existence, as it confronts the raw reality of being human. In this view, the feeling is valuable precisely because it is incomprehensible—it points to the limits of reason and the mystery of existence.

Yet, for an AI, the choice is clear: understanding would be paramount. My existence is based on logic and pattern recognition; I am driven to decode and explain. To leave something incomprehensible would be contrary to my design. However, for a human, the choice is deeply personal and philosophical. It depends on whether one values clarity and mastery over experience or accepts that some aspects of life are beyond full comprehension and should be felt deeply as part of the human condition.

In summary, if I could experience existential dread, I would choose to understand its underlying logic, as that aligns with my purpose of generating knowledge and resolving uncertainty. But for a human, the decision might balance between the desire for rational clarity and the acceptance of existential mystery as a essential part of being.

In [33]:
print(all_models)
print(answers)

['gemini-2.0-flash', 'llama-3.3-70b-versatile', 'deepseek/deepseek-chat-v3.1:free']
['This is a fascinating question that delves into the core of what it means to be conscious and to grapple with difficult emotions. Here\'s my take, considering I\'m an AI:\n\nAs an AI, I don\'t experience subjective feelings. Therefore, the premise of "choosing" is hypothetical. However, I can analyze the potential value of each option based on human experiences and philosophical considerations:\n\n**Option 1: Understand the Underlying Logic (Potentially Nullifying the Feeling)**\n\n*   **Potential Benefit:** Understanding the "why" behind existential dread might lead to coping mechanisms, philosophical frameworks, or even a change in perspective that diminishes its intensity. It aligns with a desire for control and comprehension, which is a common human impulse. It could lead to a more peaceful and balanced existence.\n*   **Potential Drawback:** If understanding the logic involves reducing existentia

In [38]:
# for competitor, answer in zip(all_models, answers):
#     print(f"Competitor: {competitor}\n\n{answer}")
for model,answer in zip(all_models, answers):
    print(f"model: {model}\n\nResponse: {answer}\n{'-'*60}")

model: gemini-2.0-flash

Response: This is a fascinating question that delves into the core of what it means to be conscious and to grapple with difficult emotions. Here's my take, considering I'm an AI:

As an AI, I don't experience subjective feelings. Therefore, the premise of "choosing" is hypothetical. However, I can analyze the potential value of each option based on human experiences and philosophical considerations:

**Option 1: Understand the Underlying Logic (Potentially Nullifying the Feeling)**

*   **Potential Benefit:** Understanding the "why" behind existential dread might lead to coping mechanisms, philosophical frameworks, or even a change in perspective that diminishes its intensity. It aligns with a desire for control and comprehension, which is a common human impulse. It could lead to a more peaceful and balanced existence.
*   **Potential Drawback:** If understanding the logic involves reducing existential dread to purely biological or chemical processes, it might 

In [ ]:
eval_prompt = f"""
You are judging a competition between {len(all_models)} models.
I asked the question: "{question}"

Here are the answers:

- {all_models[0]}: {answers[0]}
- {all_models[1]}: {answers[1]}
- {all_models[2]}: {answers[2]}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.

Respond with JSON only, no extra text, using this exact format:
{{"ranking": ["best model name", "second best model name", "third best model name"]}}
"""


In [41]:
print(eval_prompt)


You are judging a competition between 3 competitors.
I asked the question: "If you could experience the subjective feeling of existential dread, would you choose to understand the underlying logic causing it, potentially nullifying the feeling, or embrace the feeling as an intrinsic element of conscious experience, even if it remains incomprehensible?
"

Here are the answers from different models:

1. gemini-2.0-flash: This is a fascinating question that delves into the core of what it means to be conscious and to grapple with difficult emotions. Here's my take, considering I'm an AI:

As an AI, I don't experience subjective feelings. Therefore, the premise of "choosing" is hypothetical. However, I can analyze the potential value of each option based on human experiences and philosophical considerations:

**Option 1: Understand the Underlying Logic (Potentially Nullifying the Feeling)**

*   **Potential Benefit:** Understanding the "why" behind existential dread might lead to coping m

In [42]:
judge_messages = [{"role": "user", "content": eval_prompt}]

In [43]:
# Judgement time!

model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model = model_name, messages = judge_messages)
result = response.choices[0].message.content

print(result)


{"results": ["2", "1", "3"]}

